In [1]:
import os
import numpy as np
import pandas as pd
import collections
import torch
import networkx as nx
from ortools.sat.python import cp_model
from matplotlib import pyplot as plt

import warnings
warnings.filterwarnings("ignore")
from ortools_scheduler import ORtools_scheduler

#### ortools 实验结果
max_time = 300s

In [2]:
instances = ["ft06", "la01", "la06", "la11", "la21", "la31", "la36", "orb01", "swv01", "swv06", "swv11", "yn1"]

In [3]:
# 用ortools优化，保存调度结果
for instance in instances:
    scheduler = ORtools_scheduler(instance_name=instance, max_time=300)
    scheduler.optimize()
    assert scheduler.obj_val == scheduler.compute_makespan()
    print(f"For instance {instance}, obj_val in 300s is {scheduler.obj_val}")
    scheduler.store_solution()

For instance ft06, obj_val in 300s is 55.0
For instance la01, obj_val in 300s is 666.0
For instance la06, obj_val in 300s is 926.0
For instance la11, obj_val in 300s is 1222.0
For instance la21, obj_val in 300s is 1046.0
For instance la31, obj_val in 300s is 1784.0
For instance la36, obj_val in 300s is 1268.0
For instance orb01, obj_val in 300s is 1059.0
For instance swv01, obj_val in 300s is 1413.0
For instance swv06, obj_val in 300s is 1760.0
For instance swv11, obj_val in 300s is 3255.0
For instance yn1, obj_val in 300s is 898.0


In [4]:
# 读取调度结果，计算最大加工时间
for instance in instances:
    scheduler = ORtools_scheduler(instance_name=instance, max_time=300)
    scheduler.read_solution()
    makespan = scheduler.compute_makespan()
    print(f"For instance {instance}, makespan computed is {makespan}")

For instance ft06, makespan computed is 55
For instance la01, makespan computed is 666
For instance la06, makespan computed is 926
For instance la11, makespan computed is 1222
For instance la21, makespan computed is 1046
For instance la31, makespan computed is 1784
For instance la36, makespan computed is 1268
For instance orb01, makespan computed is 1059
For instance swv01, makespan computed is 1413
For instance swv06, makespan computed is 1760
For instance swv11, makespan computed is 3255
For instance yn1, makespan computed is 898
